# OHCL - Analytics

### Import Library

In [80]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import math

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.constrained_layout.use'] = True
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [81]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [82]:
data_tmp = dataset.copy()
def bar9h(tick):
    tick=tick[(tick.index.hour == 9) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]
def bar14h(tick):
    tick=tick[(tick.index.hour == 14) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]
data_tmp['Close_9h'] = data_tmp['Close']
data_tmp['Close_14h'] = data_tmp['Close']
day_data = data_tmp.resample('D').agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Close_9h': bar9h,
        'Close_14h': bar14h
    })
day_data.dropna(subset=['Close'], inplace=True)
day_data['Close_shift'] = day_data['Close'].shift(1)
day_data['Close_14h_shift'] = day_data['Close_14h'].shift(1)
day_data['Day_High_shift'] = day_data['High'].shift(1)
day_data['Day_Low_shift'] = day_data['Low'].shift(1)
day_data = day_data[['Close', 'High', 'Low', 'Close_shift', 'Close_14h_shift', 'Day_High_shift', 'Day_Low_shift']].rename(columns={'Close': 'Day_Close', 'High': 'Day_High', 'Low': 'Day_Low'})

In [83]:
day_data

,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift
Date,,,,,,,
2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN
2018-08-14,959.3,961.8,952.1,954.2,949.5,954.6,942.3
2018-08-15,946.6,962.3,946.6,959.3,960.8,961.8,952.1
2018-08-16,947.0,947.3,935.5,946.6,957.7,962.3,946.6
2018-08-17,947.0,954.6,945.0,947.0,944.0,947.3,935.5
...,...,...,...,...,...,...,...
2024-07-29,1287.8,1294.7,1287.8,1285.0,1278.4,1285.0,1276.8
2024-07-30,1292.4,1292.4,1284.2,1287.8,1292.9,1294.7,1287.8
2024-07-31,1304.3,1304.3,1291.1,1292.4,1284.9,1292.4,1284.2


In [84]:
data = dataset.copy()
data = data.assign(time_d=pd.PeriodIndex(data.index, freq='1D').to_timestamp())
data = pd.merge(data, day_data, left_on="time_d", right_index=True, how="left")

In [85]:
data = data[data.High != data.Low]
def get_percent(r):
    return (r['Close'] - r['Day_Low']) / (r['Day_High'] - r['Day_Low'])
data['percent'] = data.apply(lambda r: get_percent(r), axis=1)

In [86]:
data

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent
Date,,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.065041
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.081301
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.065041
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.024390
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,2018-08-13,954.2,954.6,942.3,NaN,NaN,NaN,NaN,0.113821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:05:00,1269.2,1273.4,1269.2,1270.8,22701,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.683962
2024-08-02 14:10:00,1270.9,1271.6,1268.9,1271.2,12412,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.702830
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.867925


In [87]:
data['last_close_rate'] = 1000*(data.Close - data.Close_shift) / data.Close_shift
data['last_highlow_rate'] = (data.Close - data.Close_14h_shift) /  (data.Day_High_shift - data.Day_Low_shift)

In [88]:
data.dropna(inplace=True)

In [89]:
data_test = data.copy()
data_test = data_test[data_test.index > '2024-01-01 00:00:00']
### Ignore this year data => tranh over fitting
data = data[data.index < '2024-01-01 00:00:00']

In [90]:
"{} - {} - {}".format(len(data[data.percent < 0.2]), len(data[data.percent > 0.8]), len(data[(data.percent > 0.2) & (data.percent < 0.8)]))

'8156 - 10644 - 47007'

In [91]:
data = data.drop(data[(data.percent > 0.2) & (data.percent < 0.8)].sample(frac=.75, random_state=1).index)

In [92]:
"{} - {} - {}".format(len(data[data.percent < 0.2]), len(data[data.percent > 0.8]), len(data[(data.percent > 0.2) & (data.percent < 0.8)]))

'8156 - 10644 - 11752'

In [93]:
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics

In [94]:
feature_cols = ["last_close_rate", "last_highlow_rate"]
# Splitting 
X_train = data[feature_cols]
y_train = data[['percent']]
X_test = data_test[feature_cols]
y_test = data_test[['percent']]

In [95]:
%%time

parameters = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [8],
    'gamma': [0.01, 0.02],
    'random_state': [42],
    'seed': [5, 100, 150]
}

eval_set = [(X_train, y_train), (X_test, y_test)]
# Create XGBClassifier model
model_xgb = xgb.XGBRegressor(eval_set=eval_set, objective ='reg:linear', verbose=False)
clf = GridSearchCV(model_xgb, parameters)

clf.fit(X_train, y_train)

CPU times: user 2min 16s, sys: 944 ms, total: 2min 17s
Wall time: 23.3 s


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    eval_set=[(                     last_close_rate  last_highlow_rate
Date                                                   
2018-08-14 09:45:00         2.200796           0.552846
2018-08-14 10:10:00         2.410396           0.569106
2018-08...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'gamma': [0.01, 0.02], 'learning_rate': [0.01, 0.05],
                         'max_depth': [8], 'n_estimators': [100, 150, 200],
                         'random_state': [42], 'seed': [5, 100, 150]})

In [96]:
print(f'Best params: {clf.best_params_}')
print(f'Best validation score = {clf.best_score_}')

Best params: {'gamma': 0.02, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 100, 'random_state': 42, 'seed': 5}
Best validation score = 0.06510467259726568


In [97]:
best_model = xgb.XGBRegressor(**clf.best_params_, objective ='reg:linear')
best_model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.02, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [98]:
y_pred = best_model.predict(X_test)

In [99]:
from sklearn.metrics import mean_squared_error
print(f'mean_squared_error = {mean_squared_error(y_test, y_pred)}')

mean_squared_error = 0.05467433023860343


In [100]:
X_result = X_test.copy()
X_result = X_result.assign(Predicts=y_pred)
X_result = X_result[['Predicts']]
data_test = pd.merge(data_test, X_result, left_index=True, right_index=True, how="left")
data_test.dropna(inplace=True)

In [101]:
data_test

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,
2024-01-02 09:00:00,1138.5,1139.7,1138.5,1138.9,6206,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.731481,3.789882,0.813953,0.528833
2024-01-02 09:05:00,1138.9,1139.0,1138.6,1139.0,1424,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.740741,3.878019,0.825581,0.528833
2024-01-02 09:10:00,1139.0,1141.5,1138.9,1141.4,6265,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.962963,5.993302,1.104651,0.636833
2024-01-02 09:15:00,1141.3,1141.8,1140.3,1140.8,3739,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.907407,5.464481,1.034884,0.662758
2024-01-02 09:20:00,1140.7,1141.2,1140.2,1140.2,1943,2024-01-02,1133.5,1141.8,1131.0,1134.6,1131.9,1139.5,1130.9,0.851852,4.935660,0.965116,0.587425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:05:00,1269.2,1273.4,1269.2,1270.8,22701,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.683962,-3.294118,-0.360795,0.467786
2024-08-02 14:10:00,1270.9,1271.6,1268.9,1271.2,12412,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.702830,-2.980392,-0.349432,0.486062
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,2024-08-02,1275.9,1277.5,1256.3,1275.0,1283.5,1306.2,1271.0,0.867925,-0.235294,-0.250000,0.512219


In [102]:
data_test[(data_test.percent > 0.8) & (data_test.Predicts > 0.8)]

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,


In [103]:
data_test[(data_test.percent < 0.2) & (data_test.Predicts < 0.2)]

,Open,High,Low,Close,Volume,time_d,Day_Close,Day_High,Day_Low,Close_shift,Close_14h_shift,Day_High_shift,Day_Low_shift,percent,last_close_rate,last_highlow_rate,Predicts
Date,,,,,,,,,,,,,,,,,
